# environment construction

In [1]:
infile_name = "20190911_8genre_288video_text"

!pip install japanize_matplotlib
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# URLを開いて認証コードをコピーしボックスに貼り付けしてください
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!cp -f ~/.gdfuse/default/config config
!sed -i -e "s/^root_folder=$/root_folder=1Dm6GSlI06zTsmRg6KdpM2WBYU0wj-Umh/" config

!mkdir -p drive
!google-drive-ocamlfuse -config ./config -cc drive
!ls drive

     |████████████████████████████████| 4.1MB 2.8MB/s 
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.0.5-cp36-none-any.whl size=4118721 sha256=6feb227b3ca3998c4944b2c500c22ddcdf0bf12df7828fb20ad48485590a4496
  Stored in directory: /root/.cache/pip/wheels/6c/8a/08/4a784957da9f3c2b4839b4986be2fba2a481877318948be52c
Successfully built japanize-matplotlib
E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 135004 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.14-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.14-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.14-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleuserc

In [2]:
# mecabインストール
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab

# mecab-ipadic-NEologdのダウンロード
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n

# 1.sedコマンドを使て、ファイルを変更、ファイル名を「mecabrc.new」として保存する。
!sed -e "s!/var/lib/mecab/dic/debian!/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd!g" /etc/mecabrc > /etc/mecabrc.new

# 4.「mecabrc」のバックアップの保存（mecabrc.orgとする）
# 3.「mecabrc.new」を「mecabrc」に上書き

!cp /etc/mecabrc /etc/mecabrc.org
!cp /etc/mecabrc.new /etc/mecabrc

!pip install mecab-python3
!pip install japanize_matplotlib

import os
# ユーザー辞書のコンパイル
os.chdir('drive/input/user_dic/')
!/usr/lib/mecab/mecab-dict-index -d /usr/share/mecab/dic/ipadic/ -u user.dic -f utf-8 -t utf-8 user.csv
!cp -p user.* /usr/share/mecab/dic/ipadic/

#「mecabrc」の中身を入れ替え
!cp mecabrc.usr /etc/mecabrc
!cat /etc/mecabrc


Reading package lists...
Building dependency tree...
Reading state information...
curl is already the newest version (7.58.0-2ubuntu3.8).
git is already the newest version (1:2.17.1-1ubuntu0.5).
sudo is already the newest version (1.8.21p2-3ubuntu1.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1 libmecab2 mecab-ipadic mecab-jumandic
  mecab-jumandic-utf8 mecab-utils
The following NEW packages will be installed:
  file libmagic-mgc libmagic1 libmecab-dev libmecab2 mecab mecab-ipadic
  mecab-ipadic-utf8 mecab-jumandic mecab-jumandic-utf8 mecab-utils
  python-mecab
0 upgraded, 12 newly installed, 0 to remove and 19 not upgraded.
Need to get 29.3 MB of archives.
After this operation, 282 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubun

In [0]:
import MeCab

import re
import csv
import datetime
import contextlib
import functools

import numpy as np
import pandas as pd
import urllib.request

import japanize_matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
sns.set(font='IPAexGothic')

from IPython.display import display, Image
import warnings
warnings.filterwarnings('ignore')
os.chdir("../../")

In [0]:
visions_df_web = pd.read_csv(f'midway/objects_name_comp_280files_2019-11-15_clean_clean_jpn.csv')
visions_df_label = pd.read_csv(f'midway/label_objects_name_comp_280files_2019-11-21_clean_jpn.csv')

In [0]:
visions_df_web = visions_df_web.rename(columns={'Unnamed: 0': 'file_name'})
visions_df_web = visions_df_web.set_index('file_name')

visions_df_label = visions_df_label.rename(columns={'Unnamed: 0': 'file_name'})
visions_df_label = visions_df_label.set_index('file_name')

In [0]:
visions_df = pd.read_csv(f'midway/objects_name_comp_280files_2019-11-15_clean.csv')
subtitles_df = pd.read_csv(f'midway/words_comp_280files_2019-11-15.csv')
labels_df = pd.read_csv(f'midway/genre_labels_comp_280files_2019-11-15.csv')

In [0]:
# NOTE: visions_df, subtitles_dfはデータ数が同じかつ、同行番号のデータは対応しているものとします
n_samples = len(visions_df)

visions_df = visions_df.rename(columns={'Unnamed: 0': 'file_name'})
visions_df = visions_df.set_index('file_name')

raw_subtitles = []
for i in range(len(subtitles_df)):
  doc = []
  for line in subtitles_df.iloc[i, 1:].values:
    if type(line) in [float, np.float64]:
      continue
    doc.append(str(line))
  raw_subtitles.append(doc)
subtitles_df = pd.DataFrame({'file_name': subtitles_df['Unnamed: 0'].values})
subtitles_df = subtitles_df.set_index('file_name')

labels_df = labels_df.set_index('file_name')

In [0]:
visions_df_web = visions_df_web.sort_index()
visions_df_label = visions_df_label.sort_index()
visions_df = visions_df.sort_index()
subtitles_df = subtitles_df.sort_index()
labels_df = labels_df.sort_index()

In [0]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels_df.genre.values)

In [10]:
import sys
sys.path.append('/content/drive/notebook/NLP')
from vectorizer import Vectorizer, SCDV, SWEM
from utility import FeatureComposition
from extractor import Extractor

Using TensorFlow backend.


File already exists.


# get X_visions

In [0]:
import re
X_visions = [[] for _ in range(n_samples)]
n_col_visions = -1
for i in range(n_samples):
  v = visions_df.iloc[i, 1:]
  for j, w in enumerate(v):
    if type(w) in [float, np.float64]:
      continue
    if re.search('^[0-9]+(\.[0-9]+)?$', w)==None: # NOTE: ここの処理は時期にいらなくなる
      X_visions[i].append(w)
  n_col_visions = max(n_col_visions, len(X_visions[i]))

for i in range(n_samples):
  X_visions[i] += [''] * (n_col_visions - len(X_visions[i]))

In [0]:
X_visions_web = [[] for _ in range(n_samples)]
n_col_visions_web = -1
for i in range(n_samples):
  v = visions_df_web.iloc[i, 1:]
  for j, w in enumerate(v):
    if type(w) in [float, np.float64]:
      continue
    if re.search('^[0-9]+(\.[0-9]+)?$', w)==None: # NOTE: ここの処理は時期にいらなくなる
      X_visions_web[i].append(w)
  n_col_visions_web = max(n_col_visions_web, len(X_visions_web[i]))
  
for i in range(n_samples):
  X_visions_web[i] += [''] * (n_col_visions_web - len(X_visions_web[i]))

In [0]:
X_visions_label = [[] for _ in range(n_samples)]
n_col_visions_label = -1
for i in range(n_samples):
  v = visions_df_label.iloc[i, 1:]
  for j, w in enumerate(v):
    if type(w) in [float, np.float64]:
      continue
    if re.search('^[0-9]+(\.[0-9]+)?$', w)==None: # NOTE: ここの処理は時期にいらなくなる
      X_visions_label[i].append(w)
  n_col_visions_label = max(n_col_visions_label, len(X_visions_label[i]))

for i in range(n_samples):
  X_visions_label[i] += [''] * (n_col_visions_label - len(X_visions_label[i]))

# get X_subtitiles

In [0]:
ex = Extractor(raw_subtitles)
X_subtitles = ex.extract([
# FIXME: 全て空配列が入っているのでdtype=float, shape=(n_samples, 1)となって配列の連結時にエラーが出ます
              # 'その他', 
              'フィラー', 
              '副詞', 
              '助動詞', 
              '助詞', 
              '動詞', 
              '名詞', 
              '形容詞', 
              '感動詞', 
              '接続詞', 
              '接頭詞', 
              '記号', 
              '連体詞'
            ])

In [0]:
class SWEM_with_ps_rate(vectorizer.Vectorizer):
  def __init__(self, col1):
    self.col1 = col1
  def fit(self, X, y=None):
    return self
  def transform(self, X):
    X = np.array(X).astype('U32')
    tr1 = vectorizer.SWEM().transform(X[:, :self.col1])
    tr2 = X[:, self.col1:].astype(float)
    return np.hstack([tr1, tr2])


# [TEST] feature selection

In [0]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_subtitles_raw)
tfidf_matrix = tokenizer.texts_to_matrix(X_subtitles_raw, "tfidf")

In [0]:
from sklearn.ensemble import RandomForestClassifier
tree = RandomForestClassifier().fit(tfidf_matrix, labels)
feature_importances = tree.feature_importances_[1:]

for i in np.argsort(-feature_importances)[:100]:
  print(tokenizer.index_word[i+1])

In [0]:
from sklearn.linear_model import RidgeClassifier
ridge = RidgeClassifier().fit(tfidf_matrix, labels)
feature_importances = ridge.coef_[:, 1:]

for i in np.argsort(-np.abs(feature_importances[1]))[:100]:
  print(tokenizer.index_word[i+1])

In [0]:
count = 0
for i, doc in enumerate(X_subtitles):
  if 'Shadowverse' in doc:
    count += 1
    print(labels_df.genre[i])
count

In [0]:
feature_importances[:, tokenizer.word_index['shadowverse']]

# supervised learning

In [15]:
from sklearn.pipeline import Pipeline
!pip install lightgbm
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.decomposition import PCA, TruncatedSVD, FactorAnalysis
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.cluster import DBSCAN, KMeans, AgglomerativeClustering

In [0]:
visions_max = Pipeline([
                    ("swem", SWEM(pooling="max")),
                    ("decomp", PCA(210)),
                    ("classifier", lgb.LGBMClassifier(random_state=0, n_estimators=100))
    ])

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_visions_web, labels, test_size=70, random_state=0)
visions_max.fit(X_train, y_train).score(X_test, y_test)

100%|██████████| 70/70 [00:04<00:00, 15.19it/s]


0.8142857142857143

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_visions_label, labels, test_size=70, random_state=0)
visions_max.fit(X_train, y_train).score(X_test, y_test)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_visions, labels, test_size=70, random_state=0)
visions_max.fit(X_train, y_train).score(X_test, y_test)

In [0]:
visions_average = Pipeline([
                    ("swem", SWEM(pooling="average")),
                    ("decomp", PCA(210)),
                    ("classifier", lgb.LGBMClassifier(random_state=0, n_estimators=100))
    ])

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_visions_web, labels, test_size=70, random_state=0)
visions_average.fit(X_train, y_train).score(X_test, y_test)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_visions_label, labels, test_size=70, random_state=0)
visions_average.fit(X_train, y_train).score(X_test, y_test)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_subtitles, labels, test_size=70, random_state=0)
average_swem.fit(X_train, y_train).score(X_test, y_test)
# 0.6428571428571429
# 0.6857142857142857 divided into lines

In [0]:
from collections import Counter
counter = Counter([w for doc in X_subtitles for w in doc if w!=''])

In [0]:
sorted_counts = np.sort(list(counter.values()))

In [0]:
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111)
counts = sorted_counts
ax.plot(np.arange(len(counts)), np.log(counts))
ax.set_xlabel('単語インデックス')
ax.set_ylabel('出現回数の対数')
plt.show()

In [0]:
sorted_counts[15000]

In [0]:
len(sorted_counts)

In [0]:
index = 60
print(sorted_counts[index])
print(list(counter.keys())[np.argsort(list(counter.values()))[index]])

In [0]:
labels_bincount = np.bincount(labels)
keywords = [[] for _ in range(7)]
for data in counter.most_common():
    word, count = data
    if count<10:
        break
    genre_count = np.zeros(7)
    for i, doc in enumerate(X_subtitles):
        genre_count[labels[i]]+= doc.count(word)
    genre_count /= labels_bincount
    genre_count /= np.linalg.norm(genre_count)
    if np.max(genre_count) >= 0.9:
        keywords[np.argmax(genre_count)].append(word)

In [0]:
for i, kw in enumerate(keywords):
    print(label_encoder.inverse_transform([i])[0])
    print(kw)

In [0]:
topic_words = ["料理", "音楽"]
doc_distances = [[] for _ in topic_words]
corpus = word_vectors.vocab.keys()
for i, tw in enumerate(topic_words):
    tw_vec = word_vectors[tw]
    for doc in X_subtitles:
        distances = []
        for word in doc:
            if not word in corpus:
                continue
            word_vec = word_vectors[word]
            dis = np.linalg.norm(word_vec-tw_vec)
            distances.append(dis)
        distances = np.sort(distances)[::-1]
        doc_distances[i].append(np.mean(distances))

In [0]:
np.array(raw_subtitles)[np.argsort(doc_distances[0])[:10]]

In [0]:
np.array(raw_subtitles)[np.array(labels)==6]